In [1]:
import pandas as pd 
import wandb
from tqdm import tqdm
import requests
import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.colors as mcolors

In [2]:
BASE_METRIC = "accuracy_per_mean_user_and_bot"
WANDB_OR_USERNAME = "guylororg"

In [3]:
def result_metric(sweeps, group_name, drop_list=[], drop_HPT=False, metric=BASE_METRIC, epoch="best"):
    df = api.get_sweeps_results(sweeps, metric=metric) 

    config_cols = [c for c in df.columns if "config_" in c and c!="config_wandb_run_id" and c!="config_online_simulation_size"]
    HPT_cols = [col for col in config_cols if df[col].nunique() > 1]
    # print(HPT_cols)
    if drop_HPT:
        df=df.drop([c for c in HPT_cols if not c in ["config_LLM_SIM_SIZE", "config_seed"]], axis=1)
        HPT_cols = ["config_LLM_SIM_SIZE", "config_seed"]
    # display(df['config_online_simulation_factor'])
    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    grouped = df.groupby([c for c in HPT_cols if c != "config_seed"])
    
    mean_df = grouped[numeric_cols].mean()
    std_df = grouped[numeric_cols].std()
    # display(mean_df)

    if epoch=="best":
        best_col = mean_df[[c for c in mean_df.columns if (metric in c and metric[-4:] == c.split("_epoch")[0][-4:])]].idxmax(axis=1)
    else:
        best_col = mean_df[[c for c in mean_df.columns if f"{metric}_epoch{epoch}" in c]].idxmax(axis=1)
    
    result = grouped.apply(lambda x: x[best_col.loc[x.name]].values)
    means = grouped.apply(lambda x: x[best_col.loc[x.name]].mean())
    stds = grouped.apply(lambda x: x[best_col.loc[x.name]].std())

    # display(result)


    df_cols = {'mean': means, 'std': stds, 'values': result.values}
    # display(df_cols)
    if epoch == "best": df_cols['epoch'] = best_col.apply(lambda x: int(x.split("epoch")[1]) if "epoch" in x else "last")

    df_cols['CI'] = result.apply(lambda x: bootstrap_ci(x))
    # display(df_cols)
    summary_df = pd.DataFrame(df_cols, index=best_col.index)
    # display(summary_df)
    for d in drop_list:
        if d in summary_df.index:
            summary_df=summary_df.drop(d)
    # if len(summary_df.index.names) == 1:
    #     return summary_df.rename_axis(group_name)
    # else:
    #     return summary_df

    return summary_df

def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
    bootstrapped_means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        bootstrapped_means.append(np.mean(sample))
    lower_bound = np.percentile(bootstrapped_means, (1 - ci) / 2 * 100)
    upper_bound = np.percentile(bootstrapped_means, (1 + ci) / 2 * 100)
    return lower_bound, upper_bound


# Results of HyperParameterTuning

For every configuration that you test in the sweep, you will receive in the table the average, standard deviation, all the values obtained for the different seed values, and also the confidence interval within which the result is located at a confidence level of 95%.

When epoch="best" is defined, you can check in which epoch the best result is obtained. If epoch=5 is defined, you will receive the result obtained for epoch number 5.

You can test multiple sweeps simultaneously by entering them into the list found in the first element of the function result_metric.

In [4]:
from read_wandb import wandb_results
api = wandb_results("NLP2024_PROJECT_Guy-lor", wandb_username=WANDB_OR_USERNAME)
sweep_ids_list = ["muwcnrjn", "8nixkte7"] # HPT==True

In [5]:
sweep_results = result_metric(sweep_ids_list, "config_online_simulation_factor",drop_list=[], drop_HPT=False, epoch="best")

Total number of sweeps: 2
Download sweep_id='muwcnrjn' data...
Download sweep_id='8nixkte7' data...


In [6]:
df = sweep_results
mapping_dict = {
    17: 'ORIGINAL PAPER STRATEGIES (OP)',
    18: 'OP + SENTIMENT RATIO',
    19: 'OP + LENGTH RATIO',
    20: 'OP + USER PREFERENCE VECTORS',
    21: 'OP + SENTIMENT RATIO + LENGTH RATIO',
    22: 'OP + SENTIMENT RATIO + LENGTH RATIO + USER PREFERENCE VECTORS',
    23: 'OP + USER PREFERENCE VECTORS (Double Weight)',
    24: 'OP + SENTIMENT RATIO + LENGTH RATIO (Double Weight)',
    25: 'OP + SENTIMENT RATIO + LENGTH RATIO + USER PREFERENCE VECTORS (Double Weight)',
    26: 'ONLY USER PREFERENCE VECTORS'
}

# Rename the MultiIndex using the mapping dictionary
df = df.rename(index=mapping_dict, level='config_basic_nature')
df

mean  \
config_basic_nature                                config_simulation_user_improve             
ORIGINAL PAPER STRATEGIES (OP)                     0.01                            0.797830   
                                                   0.05                            0.797222   
OP + SENTIMENT RATIO                               0.01                            0.794078   
                                                   0.05                            0.796739   
OP + LENGTH RATIO                                  0.01                            0.796034   
                                                   0.05                            0.795034   
OP + USER PREFERENCE VECTORS                       0.01                            0.794751   
                                                   0.05                            0.796727   
OP + SENTIMENT RATIO + LENGTH RATIO                0.01                            0.797027   
                                                   0.05                            0.793067   
OP + SENTIMENT RATIO + LENGTH RATIO + USER PREF... 0.01                            0.797775   
                                                   0.05                            0.793826   
OP + USER PREFERENCE VECTORS (Double Weight)       0.01                            0.791676   
                                                   0.05                            0.793638   
OP + SENTIMENT RATIO + LENGTH RATIO (Double Wei... 0.01                            0.796430   
                                                   0.05                            0.795526   
OP + SENTIMENT RATIO + LENGTH RATIO + USER PREF... 0.01                            0.793598   
                                                   0.05                            0.793305   
ONLY USER PREFERENCE VECTORS                       0.01                            0.785745   
                                                   0.05                            0.787624   

                                                                                        std  \
config_basic_nature                                config_simulation_user_improve             
ORIGINAL PAPER STRATEGIES (OP)                     0.01                            0.007891   
                                                   0.05                            0.008552   
OP + SENTIMENT RATIO                               0.01                            0.011826   
                                                   0.05                            0.008835   
OP + LENGTH RATIO                                  0.01                            0.007499   
                                                   0.05                            0.010897   
OP + USER PREFERENCE VECTORS                       0.01                            0.008892   
                                                   0.05                            0.007821   
OP + SENTIMENT RATIO + LENGTH RATIO                0.01                            0.006630   
                                                   0.05                            0.012446   
OP + SENTIMENT RATIO + LENGTH RATIO + USER PREF... 0.01                            0.012423   
                                                   0.05                            0.006733   
OP + USER PREFERENCE VECTORS (Double Weight)       0.01                            0.009218   
                                                   0.05                            0.010816   
OP + SENTIMENT RATIO + LENGTH RATIO (Double Wei... 0.01                            0.008464   
                                                   0.05                            0.011730   
OP + SENTIMENT RATIO + LENGTH RATIO + USER PREF... 0.01                            0.011938   
                                                   0.05                            0.007097   
ONLY USER PREFERENCE VECTORS                       0.01                            0.010318   
 

In [7]:
df[['mean','CI']]

mean  \
config_basic_nature                                config_simulation_user_improve             
ORIGINAL PAPER STRATEGIES (OP)                     0.01                            0.797830   
                                                   0.05                            0.797222   
OP + SENTIMENT RATIO                               0.01                            0.794078   
                                                   0.05                            0.796739   
OP + LENGTH RATIO                                  0.01                            0.796034   
                                                   0.05                            0.795034   
OP + USER PREFERENCE VECTORS                       0.01                            0.794751   
                                                   0.05                            0.796727   
OP + SENTIMENT RATIO + LENGTH RATIO                0.01                            0.797027   
                                                   0.05                            0.793067   
OP + SENTIMENT RATIO + LENGTH RATIO + USER PREF... 0.01                            0.797775   
                                                   0.05                            0.793826   
OP + USER PREFERENCE VECTORS (Double Weight)       0.01                            0.791676   
                                                   0.05                            0.793638   
OP + SENTIMENT RATIO + LENGTH RATIO (Double Wei... 0.01                            0.796430   
                                                   0.05                            0.795526   
OP + SENTIMENT RATIO + LENGTH RATIO + USER PREF... 0.01                            0.793598   
                                                   0.05                            0.793305   
ONLY USER PREFERENCE VECTORS                       0.01                            0.785745   
                                                   0.05                            0.787624   

                                                                                                                         CI  
config_basic_nature                                config_simulation_user_improve                                            
ORIGINAL PAPER STRATEGIES (OP)                     0.01                            (0.7915834810644624, 0.8040770675689211)  
                                                   0.05                             (0.790211285893607, 0.8042332410528391)  
OP + SENTIMENT RATIO                               0.01                            (0.7845309625659587, 0.8036252475635322)  
                                                   0.05                            (0.7896887050433703, 0.8037897838251682)  
OP + LENGTH RATIO                                  0.01                            (0.7895819611508536, 0.8024865517995263)  
                                                   0.05                            (0.7856184450249026, 0.8044504378740028)  
OP + USER PREFERENCE VECTORS                       0.01                            (0.7878448898600064, 0.8020106568325495)  
                                                   0.05                            (0.7907452223409499, 0.8032818885771055)  
OP + SENTIMENT RATIO + LENGTH RATIO                0.01                             (0.790424099515907, 0.8015301047786629)  
                                                   0.05                            (0.7840843822418442, 0.8055013052767528)  
OP + SENTIMENT RATIO + LENGTH RATIO + USER PREF... 0.01                            (0.7877227858116433, 0.8078280746744186)  
                                                   0.05                            (0.7883121931927827, 0.7993392519003312)  
OP + USER PREFERENCE VECTORS (Double Weight)       0.01                            (0.7842900885953421, 0.7991117467529666)  
                                                   0.05                            (0.7869720987617671, 0.

# Result on Test

In [8]:
sweep_ids_list = ['2gmspxn7'] # HPT==False 
sweep_results = result_metric(sweep_ids_list, "config_online_simulation_factor",drop_list=[], drop_HPT=False, epoch="best")

Total number of sweeps: 1
Download sweep_id='2gmspxn7' data...


In [9]:
df = sweep_results
mapping_dict = {
    17: 'ORIGINAL PAPER STRATEGIES (OP)',
    18: 'OP + SENTIMENT RATIO',
    19: 'OP + LENGTH RATIO',
    20: 'OP + USER PREFERENCE VECTORS',
    21: 'OP + SENTIMENT RATIO + LENGTH RATIO',
    22: 'OP + SENTIMENT RATIO + LENGTH RATIO + USER PREFERENCE VECTORS',
    23: 'OP + USER PREFERENCE VECTORS (Double Weight)',
    24: 'OP + SENTIMENT RATIO + LENGTH RATIO (Double Weight)',
    25: 'OP + SENTIMENT RATIO + LENGTH RATIO + USER PREFERENCE VECTORS (Double Weight)',
    26: 'ONLY USER PREFERENCE VECTORS'
}

# Rename the MultiIndex using the mapping dictionary
df = df.rename(index=mapping_dict, level='config_basic_nature')
df

,mean,std,values,epoch,CI
config_basic_nature,,,,,
ORIGINAL PAPER STRATEGIES (OP),0.835264,0.002924,"[0.8394127949732522, 0.8347798132165378, 0.831...",11,"(0.8331097398219548, 0.8378110031930424)"
OP + USER PREFERENCE VECTORS,0.835367,0.003055,"[0.8375905369015115, 0.83879266154687, 0.83258...",8,"(0.8329749392018932, 0.8377596822028203)"
ONLY USER PREFERENCE VECTORS,0.829595,0.003508,"[0.8288568696208233, 0.8255839366017164, 0.835...",11,"(0.8270860657730786, 0.8325948419073333)"


In [10]:
df[['mean','CI']]

,mean,CI
config_basic_nature,,
ORIGINAL PAPER STRATEGIES (OP),0.835264,"(0.8331097398219548, 0.8378110031930424)"
OP + USER PREFERENCE VECTORS,0.835367,"(0.8329749392018932, 0.8377596822028203)"
ONLY USER PREFERENCE VECTORS,0.829595,"(0.8270860657730786, 0.8325948419073333)"
